In [5]:
import pandas as pd
joy = pd.read_csv(r"C:\joy\accident_ data _joy.csv")

joy.head(12)

,Age,Sex,Education,Vehicle,Driving_experience,Lanes,Junction,Road,Time,Weather_conditions,Collision_type,Vehicle_movement,Pedestrian_movement,Cause_of_accident,Accident_severity
0,18-30,Male,Above high school,Employee,1-2yr,Unknown,No junction,Asphalt roads,Daylight,Normal,Collision with roadside-parked vehicles,Going straight,Not a Pedestrian,Moving Backward,2
1,31-50,Male,Junior high school,Employee,Above 10yr,Undivided Two way,No junction,Asphalt roads,Daylight,Normal,Vehicle with vehicle collision,Going straight,Not a Pedestrian,Overtaking,2
2,18-30,Male,Junior high school,Employee,1-2yr,other,No junction,Asphalt roads,Daylight,Normal,Collision with roadside objects,Going straight,Not a Pedestrian,Changing lane to the left,1
3,18-30,Male,Junior high school,Employee,5-10yr,other,Y Shape,Earth roads,Darkness - lights lit,Normal,Vehicle with vehicle collision,Going straight,Not a Pedestrian,Changing lane to the right,2
4,18-30,Male,Junior high school,Employee,2-5yr,other,Y Shape,Asphalt roads,Darkness - lights lit,Normal,Vehicle with vehicle collision,Going straight,Not a Pedestrian,Overtaking,2
5,31-50,Male,Unknown,Unknown,Unknown,Unknown,Y Shape,Unknown,Daylight,Normal,Vehicle with vehicle collision,U-Turn,Not a Pedestrian,Overloading,2
6,18-30,Male,Junior high school,Employee,2-5yr,Undivided Two way,Crossing,Unknown,Daylight,Normal,Vehicle with vehicle collision,Moving Backward,Not a Pedestrian,Other,2
7,18-30,Male,Junior high school,Employee,2-5yr,other,Y Shape,Asphalt roads,Daylight,Normal,Vehicle with vehicle collision,U-Turn,Not a Pedestrian,No priority to vehicle,2
8,18-30,Male,Junior high school,Employee,Above 10yr,other,Y Shape,Earth roads,Daylight,Normal,Collision with roadside-parked vehicles,Going straight,Crossing from driver's nearside,Changing lane to the right,2
9,18-30,Male,Junior high school,Employee,1-2yr,Undivided Two way,Y Shape,Asphalt roads,Daylight,Normal,Collision with roadside-parked vehicles,U-Turn,Not a Pedestrian,Moving Backward,1


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pickle

df = pd.read_csv(r"C:\joy\accident_ data _joy.csv")

print(df.columns)

categorical_columns = ['Age', 'Sex', 'Education', 'Vehicle', 'Lanes', 'Junction', 
                       'Road', 'Time', 'Weather_conditions', 'Collision_type', 
                       'Vehicle_movement', 'Pedestrian_movement', 'Cause_of_accident', 
                       'Driving_experience']

X = df.drop('Accident_severity', axis=1) 
y = df['Accident_severity'] 

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ], remainder='passthrough')  # Leave numerical data as is

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', LinearRegression())])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

with open('road_accident_severity_model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

with open('road_accident_severity_model.pkl', 'rb') as file:
    pipeline = pickle.load(file)

new_data = pd.DataFrame({
    'Age': ['18-30'], 
    'Sex': ['Male'], 
    'Education': ['Junior high school'],
    'Vehicle': ['Employee'],
    'Driving_experience': ['5-10yr'],
    'Lanes': ['Undivided Two way'],
    'Junction': ['No junction'],
    'Road': ['Asphalt roads'],
    'Time': ['Daylight'],
    'Weather_conditions': ['Normal'],
    'Collision_type': ['Collision with roadside-parked vehicles'],
    'Vehicle_movement': ['Going straight'],
    'Pedestrian_movement': ['Not a Pedestrian'],
    'Cause_of_accident': ['Changing lane to the left']
})

predicted_severity = pipeline.predict(new_data)
print(f'Predicted Accident Severity: {predicted_severity[0]}')


Index(['Age', 'Sex', 'Education', 'Vehicle', 'Driving_experience', 'Lanes',
       'Junction', 'Road', 'Time', 'Weather_conditions', 'Collision_type',
       'Vehicle_movement', 'Pedestrian_movement', 'Cause_of_accident',
       'Accident_severity'],
      dtype='object')
Mean Squared Error: 0.33009160171000795
Predicted Accident Severity: 1.297378513252255
